In [2]:
from wmfdata import mariadb
from wmfdata.utils import sql_tuple

from datetime import datetime
import pandas as pd
pd.options.display.max_columns = None

import re
import warnings

import requests
from bs4 import BeautifulSoup

[canonical-data](https://github.com/wikimedia-research/canonical-data/blob/master/wiki/wikis.tsv) is the best available and accessible source to gather data for list of wikis, their visilibity and status.

In [3]:
cd_wikis = pd.read_csv('https://raw.githubusercontent.com/wikimedia-research/canonical-data/master/wiki/wikis.tsv', sep='\t')

# for the scope of this analysis, we are only concerned about publicly visible and editable projects
public_dbs = cd_wikis.query("""(visibility == 'public') & (editability == 'public')""").reset_index(drop=True)

# further limiting to content projects: with this, test wikis, organizational wikis, wikimania wikis etc. will be dropped
content_db_groups = ['commons', 'wikibooks', 'wikidata', 'wikinews', 'wikipedia', 'wikiquote', 'wikisource', 'wikiversity', 'wikivoyage', 'wiktionary']
public_content_dbs = public_dbs.query("""database_group == @content_db_groups""")

## Gather git blame data
mediawiki-config files on gerrit.wikimedia.org have [lists of databases by various categories](https://gerrit.wikimedia.org/r/plugins/gitiles/operations/mediawiki-config/+/refs/heads/master/dblists), when a wiki is created/closed the database code is added to the respective list. [git-blame](https://git-scm.com/docs/git-blame) helps to see which line of the file was last modified when and by whom. As the dblists are structured as one db (wiki) per line, this data can be used to understand when a wiki was created, as database created is an essential step for wiki creation.

In [4]:
# get the blame data into text files in git_blame_data directory
# for all, closed, private, and fishbowl
!chmod +x get_blame.sh
!./get_blame.sh

Cloning into 'mediawiki-config'...
remote: Counting objects: 9, done
remote: Finding sources: 100% (9/9)
remote: Getting sizes: 100% (6/6)
remote: Compressing objects: 100% (121267/121267)
remote: Total 141772 (delta 1), reused 141766 (delta 1)2 MiB/s   
Receiving objects: 100% (141772/141772), 171.26 MiB | 34.43 MiB/s, done.
Resolving deltas: 100% (99601/99601), done.
yes: standard output: Broken pipe


In [5]:
# processing the git_blame_data from txt to dict, and to Pandas Dataframe
blame_data = {}

for category in ['all', 'closed']:
    
    blame_data[category] = {}
    
    with open(f'git_blame_data/{category}.txt') as blame_file:
        lines = blame_file.readlines()
    
    for line in lines:
        if 'Do not edit it' in line:
            pass
        else:
            wiki_db = re.sub('.*\)', '', line).strip()
            dt = re.findall('\d{4}-\d{2}-\d{2}', line)[0]
            blame_data[category][wiki_db] = dt

In [7]:
# dict to dataframe and rename columns
blame_data_df = (pd.DataFrame(blame_data)
                 .reset_index()
                 .rename({'index': 'database_code', 'all': 'git_created_dt', 'closed': 'git_closed_dt'}, axis=1))

# combine git blame data with list of content dbs
public_content_dbs = pd.merge(public_content_dbs, blame_data_df, on='database_code', how='left')

# covert required columns to datatime format
public_content_dbs['git_created_dt'] = pd.to_datetime(public_content_dbs['git_created_dt'], yearfirst=True, errors='coerce')
public_content_dbs['git_closed_dt'] = pd.to_datetime(public_content_dbs['git_closed_dt'], yearfirst=True, errors='coerce')

public_content_dbs.head()

,database_code,domain_name,database_group,language_code,language_name,status,visibility,editability,english_name,git_created_dt,git_closed_dt
0,aawiki,aa.wikipedia.org,wikipedia,aa,Afar,closed,public,public,Afar Wikipedia,2012-02-24,2012-02-24
1,aawikibooks,aa.wikibooks.org,wikibooks,aa,Afar,closed,public,public,Afar Wikibooks,2012-02-24,2012-02-24
2,aawiktionary,aa.wiktionary.org,wiktionary,aa,Afar,closed,public,public,Afar Wiktionary,2012-02-24,2012-02-24
3,abwiki,ab.wikipedia.org,wikipedia,ab,Abkhazian,open,public,public,Abkhazian Wikipedia,2012-02-24,NaT
4,abwiktionary,ab.wiktionary.org,wiktionary,ab,Abkhazian,closed,public,public,Abkhazian Wiktionary,2012-02-24,2012-02-24


In [8]:
# frequency of dates when a project was added to mediawiki_config files
print('creation:\n', public_content_dbs.git_created_dt.value_counts().head(3), '\n')
print('closed:\n', public_content_dbs.git_closed_dt.value_counts().head(3))

creation:
 2012-02-24    695
2012-05-08     72
2012-11-06      8
Name: git_created_dt, dtype: int64 

closed:
 2012-02-24    89
2012-05-16     5
2013-07-23     2
Name: git_closed_dt, dtype: int64


## wiki_db creation date

The git files were created on 2012-02-24, which is the reason for many wikis having their date recorded as 2012-02-24. This source can only be used for wikis created after 2012-02-24. For wikis created before the date, the date of first revision will be used as a proxy.

In [96]:
pre2012_wikis = public_content_dbs[public_content_dbs['git_created_dt'] <= datetime(2012, 2, 24)].reset_index(drop=True)
pre2012_wikis

,database_code,domain_name,database_group,language_code,language_name,status,visibility,editability,english_name,git_created_dt,git_closed_dt
0,aawiki,aa.wikipedia.org,wikipedia,aa,Afar,closed,public,public,Afar Wikipedia,2012-02-24,2012-02-24
1,aawikibooks,aa.wikibooks.org,wikibooks,aa,Afar,closed,public,public,Afar Wikibooks,2012-02-24,2012-02-24
2,aawiktionary,aa.wiktionary.org,wiktionary,aa,Afar,closed,public,public,Afar Wiktionary,2012-02-24,2012-02-24
3,abwiki,ab.wikipedia.org,wikipedia,ab,Abkhazian,open,public,public,Abkhazian Wikipedia,2012-02-24,NaT
4,abwiktionary,ab.wiktionary.org,wiktionary,ab,Abkhazian,closed,public,public,Abkhazian Wiktionary,2012-02-24,2012-02-24
...,...,...,...,...,...,...,...,...,...,...,...
690,zhwikisource,zh.wikisource.org,wikisource,zh,Chinese,open,public,public,Chinese Wikisource,2012-02-24,NaT
691,zhwiktionary,zh.wiktionary.org,wiktionary,zh,Chinese,open,public,public,Chinese Wiktionary,2012-02-24,NaT
692,zuwiki,zu.wikipedia.org,wikipedia,zu,Zulu,open,public,public,Zulu Wikipedia,2012-02-24,NaT
693,zuwikibooks,zu.wikibooks.org,wikibooks,zu,Zulu,closed,public,public,Zulu Wikibooks,2012-02-24,2012-02-24


In [97]:
incubator_pre2012_logs = pd.read_csv('incubator_site_creation_log_pre2012.csv')
incubator_pre2012_logs.head()

,domain_name
0,ace.wikipedia.org
1,ar.wikiversity.org
2,arz.wikipedia.org
3,bar.wikipedia.org
4,bcl.wikipedia.org


In [101]:
incbator_200610_logs = pd.read_csv('incubator_site_creation_log_2006-2010.csv')
incbator_200610_logs.head()

,date,domain_name
0,30 September 2006,bar.wikipedia.org
1,30 September 2006,bpy.wikipedia.org
2,30 September 2006,bxr.wikipedia.org
3,30 September 2006,cdo.wikipedia.org
4,30 September 2006,cbk-zam.wikipedia.org


In [ ]:
wikis_200610 = incubator_pre2012_logs.query("""domain_name != @incbator_200610_logs.domain_name.values.tolist()""")
wikis_200610.head()

,domain_name
1,ar.wikiversity.org
6,be.wikisource.org
7,bjn.wikipedia.org
11,br.wikisource.org
22,el.wikinews.org


In [109]:
non_incubator_creations = pre2012_wikis.query("""domain_name != @incubator_pre2012_logs.domain_name.values.tolist()""")

In [110]:
non_incubator_creations.head()

,database_code,domain_name,database_group,language_code,language_name,status,visibility,editability,english_name,git_created_dt,git_closed_dt
0,aawiki,aa.wikipedia.org,wikipedia,aa,Afar,closed,public,public,Afar Wikipedia,2012-02-24,2012-02-24
1,aawikibooks,aa.wikibooks.org,wikibooks,aa,Afar,closed,public,public,Afar Wikibooks,2012-02-24,2012-02-24
2,aawiktionary,aa.wiktionary.org,wiktionary,aa,Afar,closed,public,public,Afar Wiktionary,2012-02-24,2012-02-24
3,abwiki,ab.wikipedia.org,wikipedia,ab,Abkhazian,open,public,public,Abkhazian Wikipedia,2012-02-24,NaT
4,abwiktionary,ab.wiktionary.org,wiktionary,ab,Abkhazian,closed,public,public,Abkhazian Wiktionary,2012-02-24,2012-02-24


In [121]:
min_rev_date = wmf.mariadb.run("""SELECT MIN(rev_timestamp) AS min_rev_timestamp FROM revision""", non_incubator_creations['database_code'].values.tolist())
min_rev_date['database_code'] = non_incubator_creations['database_code'].values.tolist()
min_rev_date['min_rev_timestamp'] = pd.to_datetime(min_rev_date['min_rev_timestamp'], yearfirst=True, errors='coerce')
min_rev_date['min_rev_timestamp'] = min_rev_date['min_rev_timestamp'].apply(lambda x:x.date())
min_rev_date.head()

,min_rev_timestamp,database_code
0,2005-07-07,aawiki
1,2004-08-13,aawikibooks
2,2004-01-29,aawiktionary
3,2003-10-12,abwiki
4,2004-01-29,abwiktionary


In [161]:
def extract_messages(year, month):
    url = f'https://lists.wikimedia.org/hyperkitty/list/newprojects@lists.wikimedia.org/{year}/{month}/'
    result = requests.get(url)
    page_content = BeautifulSoup(result.content, 'html.parser')
    creation_dts = {}
    for thread in page_content.find_all('div', {'class': 'thread-email row'}):
        wiki_db = thread.find('span', class_='thread-title').text.replace('New wiki: ', '').strip()
        date = thread.find('div', class_='threa-date').get('title')
        date = datetime.strptime(date, '%A, %d %B %Y %H:%M:%S').date()
        creation_dts[wiki_db] = date
    return creation_dts

In [174]:
creation_dts = {}
for year in range(2010, 2012+1):
    for month in range(1, 12+1):
        output = extract_messages(year, month)
        creation_dts = creation_dts | output

creation_dts

{'movementroleswiki': datetime.date(2010, 7, 30),
 'noboard_chapterswikimedia': datetime.date(2010, 7, 20),
 'Tue, 20 Jul 2010 13:30:57 GMT': datetime.date(2010, 7, 20),
 'frrwiki': datetime.date(2010, 8, 24),
 'mkwikimedia': datetime.date(2010, 8, 20),
 'kowikinews': datetime.date(2010, 8, 20),
 'tenwiki': datetime.date(2010, 11, 9),
 'tenwikipedia': datetime.date(2010, 10, 26),
 '10wikipedia': datetime.date(2010, 10, 26),
 'etwikimedia': datetime.date(2010, 10, 23),
 'fawikinews': datetime.date(2010, 10, 17),
 'koiwiki': datetime.date(2010, 10, 17),
 'bjnwiki': datetime.date(2010, 10, 17),
 'mrjwiki': datetime.date(2010, 10, 17),
 'svwikiversity': datetime.date(2010, 11, 14),
 'pflwiki': datetime.date(2010, 11, 14),
 'eowikinews': datetime.date(2010, 11, 13),
 'liwikibooks': datetime.date(2010, 11, 13),
 'brwikisource': datetime.date(2010, 11, 13),
 'vecwikisource': datetime.date(2010, 11, 13),
 'gagwiki': datetime.date(2010, 11, 13),
 'ruewiki': datetime.date(2011, 1, 23),
 'test2wi

In [ ]:
creation_dt_201012 = pd.DataFrame(creation_dts.values(), index=creation_dts.keys(), columns=['date']).reset_index().rename({'index': 'database_code'}, axis=1)
creation_dt_201012

In [200]:
for i in creation_dt_201012.index:
    db_code = creation_dt_201012.loc[i, 'database_code']
    
    if re.search('\d{2}:\d{2}:\d{2}', db_code):
        creation_dt_201012.drop(i, axis=0, inplace=True)
    
    if re.search('.*..*,.*', db_code):
        dbs = db_code.split(':')[1].split(',')
        df = pd.DataFrame([i.strip() for i in dbs],  columns = ['database_code'])
        df['date'] = creation_dt_201012.loc[i, 'date']
        creation_dt_201012.drop(i, axis=0, inplace=True)
        creation_dt_201012 = pd.concat([creation_dt_201012, df], ignore_index=True)

creation_dt_201012.head()

,database_code,date
0,movementroleswiki,2010-07-30
1,noboard_chapterswikimedia,2010-07-20
2,frrwiki,2010-08-24
3,mkwikimedia,2010-08-20
4,kowikinews,2010-08-20


In [206]:
creation_dt_201012 = pd.merge(creation_dt_201012, cd_wikis[['domain_name', 'database_code']], on='database_code', how='left')

In [217]:
incubator201012 = incubator_pre2012_logs.query("""domain_name != @incbator_200610_logs.domain_name.values.tolist()""")
incubator201012 = pd.merge(incubator201012, creation_dt_201012, on='domain_name', how='left').drop('database_code', axis=1)
incubator201012

,domain_name,date
0,ar.wikiversity.org,2011-07-12
1,be.wikisource.org,2012-03-27
2,bjn.wikipedia.org,2010-10-17
3,br.wikisource.org,2010-11-13
4,el.wikinews.org,2011-04-09
5,eo.wikinews.org,2010-11-13
6,eo.wikisource.org,2011-03-20
7,fa.wikinews.org,2010-10-17
8,frr.wikipedia.org,2010-08-24
9,gag.wikipedia.org,2010-11-13


In [218]:
missing_dates = {
    'nso.wikipedia.org': datetime(2011, 10, 29).date(),
    'or.wiktionary.org': datetime(2011, 9, 28).date()
}

In [219]:
for i in incubator201012[incubator201012.date.isna()].index:
    incubator201012.loc[i, 'date'] = missing_dates[incubator201012.loc[i, 'domain_name']]

## wiki closure date
(under construction)

In [8]:
closed_content_wikis = public_content_dbs.query("""status == 'closed'""")

In [9]:
closed_content_wikis.git_closed_dt.value_counts()

2012-02-24    89
2012-05-16     5
2013-07-23     2
2014-12-17     2
2013-01-09     2
2012-09-29     2
2019-09-23     1
2021-01-13     1
2012-08-05     1
2020-06-16     1
2019-02-07     1
2012-06-10     1
Name: git_closed_dt, dtype: int64

In [10]:
db_group_prefix_map = {'wikipedia': 'Wp',   
                       'wikibooks': 'Wb',
                       'wiktionary': 'Wt', 
                       'wikiquote': 'Wq', 
                       'wikisource': 'Ws',
                       'wikinews': 'Wn',
                       'wikivoyage': 'Wy',
                       'wikiversity': 'Wv'}

In [11]:
def generate_prefix(db_group, language_code, prefix_map=db_group_prefix_map):
    return f'{db_group_prefix_map[db_group]}/{language_code}'

In [12]:
closed_content_wikis['prefix'] = closed_content_wikis[['database_group', 'language_code']].apply(lambda x:generate_prefix(x.database_group, x.language_code), axis=1)

/tmp/ipykernel_3982/1928400913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_content_wikis['prefix'] = closed_content_wikis[['database_group', 'language_code']].apply(lambda x:generate_prefix(x.database_group, x.language_code), axis=1)


In [13]:
import_log_query = """
WITH 
    logs AS (
        SELECT 
            log_id,
            log_timestamp,
            log_title,
            REGEXP_SUBSTR(log_title, 'W[a-z]/[a-z]+') AS prefix
        FROM 
            logging
        WHERE 
            log_type = 'import'
        HAVING
            REGEXP_SUBSTR(log_title, 'W[a-z]/[a-z]+') IN {CLOSED_DBS}),
    
    first_log AS (
        SELECT
            MIN(log_timestamp) AS log_timestamp,
            prefix
        FROM
            logs
        GROUP BY
            prefix)
    
SELECT
    prefix,
    CONCAT(YEAR(log_timestamp), '-', MONTH(log_timestamp), '-', DAY(log_timestamp)) AS first_log_timestamp
FROM 
    first_log
"""

incubator_import_log = wmf.mariadb.run(import_log_query.format(CLOSED_DBS=sql_tuple(closed_content_wikis.prefix.values)), dbs='incubatorwiki')
incubator_import_log.head()

/home/kcv-wikimf/.conda/envs/2023-03-31T07.02.22_kcv-wikimf/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,prefix,first_log_timestamp
0,Wb/ast,2011-9-3
1,Wb/bm,2008-5-14
2,Wb/co,2011-5-1
3,Wb/kn,2012-4-9
4,Wb/lv,2011-3-17


In [175]:
incubator_import_log.shape

(37, 2)

In [14]:
closed_content_wikis = pd.merge(closed_content_wikis, incubator_import_log, on='prefix', how='left')
closed_content_wikis.head()

,database_code,domain_name,database_group,language_code,language_name,status,visibility,editability,english_name,git_created_dt,git_closed_dt,prefix,first_log_timestamp
0,aawiki,aa.wikipedia.org,wikipedia,aa,Afar,closed,public,public,Afar Wikipedia,2012-02-24,2012-02-24,Wp/aa,2008-5-13
1,aawikibooks,aa.wikibooks.org,wikibooks,aa,Afar,closed,public,public,Afar Wikibooks,2012-02-24,2012-02-24,Wb/aa,NaN
2,aawiktionary,aa.wiktionary.org,wiktionary,aa,Afar,closed,public,public,Afar Wiktionary,2012-02-24,2012-02-24,Wt/aa,2008-8-25
3,abwiktionary,ab.wiktionary.org,wiktionary,ab,Abkhazian,closed,public,public,Abkhazian Wiktionary,2012-02-24,2012-02-24,Wt/ab,NaN
4,akwikibooks,ak.wikibooks.org,wikibooks,ak,Akan,closed,public,public,Akan Wikibooks,2012-02-24,2012-02-24,Wb/ak,NaN


In [50]:
close_missing = closed_content_wikis[closed_content_wikis.first_log_timestamp.isna()]
close_missing

,database_code,domain_name,database_group,language_code,language_name,status,visibility,editability,english_name,git_created_dt,git_closed_dt,prefix,first_log_timestamp
1,aawikibooks,aa.wikibooks.org,wikibooks,aa,Afar,closed,public,public,Afar Wikibooks,2012-02-24,2012-02-24,Wb/aa,NaN
3,abwiktionary,ab.wiktionary.org,wiktionary,ab,Abkhazian,closed,public,public,Abkhazian Wiktionary,2012-02-24,2012-02-24,Wt/ab,NaN
4,akwikibooks,ak.wikibooks.org,wikibooks,ak,Akan,closed,public,public,Akan Wikibooks,2012-02-24,2012-02-24,Wb/ak,NaN
5,akwiktionary,ak.wiktionary.org,wiktionary,ak,Akan,closed,public,public,Akan Wiktionary,2012-02-24,2012-02-24,Wt/ak,NaN
7,angwikibooks,ang.wikibooks.org,wikibooks,ang,Old English,closed,public,public,Old English Wikibooks,2012-02-24,2014-12-17,Wb/ang,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,yowiktionary,yo.wiktionary.org,wiktionary,yo,Yoruba,closed,public,public,Yoruba Wiktionary,2012-02-24,2012-02-24,Wt/yo,NaN
102,zawikibooks,za.wikibooks.org,wikibooks,za,Zhuang,closed,public,public,Zhuang Wikibooks,2012-02-24,2012-02-24,Wb/za,NaN
103,zawikiquote,za.wikiquote.org,wikiquote,za,Zhuang,closed,public,public,Zhuang Wikiquote,2012-02-24,2012-02-24,Wq/za,NaN
105,zh_min_nanwikibooks,zh-min-nan.wikibooks.org,wikibooks,zh-min-nan,Chinese (Min Nan),closed,public,public,Min Nan Wikibooks,2012-02-24,2012-09-29,Wb/zh-min-nan,NaN


In [15]:
response = requests.get('https://meta.wikimedia.org/wiki/Stewards/Former_stewards')
soup = BeautifulSoup(response.text, 'html.parser')

In [25]:
former_stewards = pd.read_html(str(soup.find('table', {'class': 'sortable'})))[0]

In [227]:
closed_content_wikis.shape

(108, 12)

In [27]:
current_stewards = wmf.mariadb.run("""
SELECT *
FROM global_user_groups ug
JOIN globaluser u
ON u.gu_id = ug.gug_user
WHERE gug_group = 'steward'


""", ['centralauth'], use_x1=True)

/home/kcv-wikimf/.conda/envs/2023-03-31T07.02.22_kcv-wikimf/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [32]:
stewards = former_stewards.Username.values.tolist() + current_stewards.gu_name.values.tolist()

In [34]:
stewards_sql = sql_tuple(stewards)

In [49]:
df = wmf.mariadb.run(f"""
WITH max_time AS (SELECT MAX(rev_timestamp) AS timestamp
FROM revision r
JOIN actor a
ON r.rev_actor = a.actor_id
WHERE NOT (actor_name IN {stewards_sql} OR actor_name IN ('Flow talk page manager', 'Global rename script', 'MediaWiki message delivery', 'Maintenance script')))

SELECT * FROM revision r
JOIN actor a ON r.rev_actor = a.actor_id
JOIN max_time m WHERE r.rev_timestamp = m.timestamp


""", ['abwiktionary'])


df




/home/kcv-wikimf/.conda/envs/2023-03-31T07.02.22_kcv-wikimf/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,rev_id,rev_page,rev_comment_id,rev_actor,rev_timestamp,rev_minor_edit,rev_deleted,rev_len,rev_parent_id,rev_sha1,actor_id,actor_user,actor_name,timestamp
0,8916,1998,130,87,20080514110341,0,0,91,0,hnstyp4d0bi1l39ketnwp7cec7q0hmr,87,86,VasilievVV,20080514110341


In [ ]:
warnings.filterwarnings('ignore')
df = wmf.mariadb.run(f"""
SELECT *, MAX(rev_timestamp) AS timestamp
FROM revision r
JOIN actor a
ON r.rev_actor = a.actor_id
WHERE actor_name NOT IN {stewards_sql}


""", close_missing.database_code.values.tolist())

df['wiki_db'] = close_missing.database_code.values.tolist()
df




In [59]:
df.actor_name.unique()

array(['', 'MediaWiki default', 'RCBot~akwiktionary', '127.0.0.1',
       '217.93.15.182', 'RCBot~bhwiktionary', 'RCBot~biwiktionary',
       '82.253.83.205', 'Zigger', 'RCBot~bowiktionary',
       'Hégésippe Cormier', 'RCBot~crwiktionary', 'RCBot~hzwiki',
       'RCBot~krwiki', '195.56.48.93', '66.201.174.92', 'Robin Patterson',
       '222.101.250.178', '217.84.101.177', 'RCBot~rnwiktionary',
       '62.11.157.231', 'Zafar', 'RCBot~towiktionary', '69.79.186.251',
       'RCBot~yowiktionary'], dtype=object)